## Probablistic Crop Yield Forecast (PCYF) model

In [6]:
import os, sys, getopt
from tools import save_hdf
import time
import numpy as np
import pandas as pd
from pcyf import PCYF
import metrics as mt

In [7]:
# Load data
crop = pd.read_hdf('./data/crop/SO_admin2_maize_yield.hdf')
dist = crop.columns
dist.name = 'FNID_ADM2'
targmon = 2
leadmat = [4,3,2,1]

# Prediction per district
msess = np.full([len(dist), 4], np.nan)
gss = msess.copy()
for i, pid in enumerate(dist):
    # Load data
    dfCrop = pd.read_hdf('./data/data_in/dfCropDist%d.hdf' % i)[pid]
    dfPred = pd.read_hdf('./data/data_in/dfPredDist%d.hdf' % i)[pid]
    # Prediction
    box = PCYF(dfCrop, dfPred, targmon=targmon, leadmat=leadmat, pid=pid)
    if box.outbox['status'] == 0:
        for j in range(4):
            msess[i,j] = box.outbox['m%02d'%leadmat[j]]['msess']
            gss[i,j] = box.outbox['m%02d'%leadmat[j]]['gss']

In [8]:
# Skill scores
msess = pd.DataFrame(index = dist, data = msess, columns = ['mse04','mse03','mse02','mse01'])
gss = pd.DataFrame(index = dist, data = gss, columns = ['gss04','gss03','gss02','gss01'])

In [9]:
save_hdf('./data/result/admin2_msess.hdf', msess)
save_hdf('./data/result/admin2_gss.hdf', gss)

./data/result/admin2_msess.hdf is saved.
./data/result/admin2_gss.hdf is saved.
